El objetivo de este código es implementar el algoritmo GBM, separando los datos por **repetición**.
Va a buscar los datos en una carpeta que se encuentre en el mismo directorio que se llame *datos_baseline*.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/repo_tesis/entorno_tesis_Molina"
!source bin/activate

Mounted at /content/drive
/content/drive/My Drive/repo_tesis/entorno_tesis_Molina


In [2]:
# instalar comet ml en caso que sea necesario
!pip install comet_ml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.9/648.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 979.1/979.1 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.8.0
    Uninstalling websocket-client-1.8.0:
      Successfully uninstalled websocket-client-1.8.0
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1


In [3]:
# Importo las librerías
from lightgbm import LGBMClassifier
import numpy as np
from joblib import load
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
import seaborn as sns
import time
from comet_ml import Experiment
import joblib
import matplotlib.pyplot as plt

In [4]:
# me situo en el directorio
%cd "/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline"

# Levanto los datos
features = load("FEATURES_W200_I50.joblib")
label = load("label_W200_I50.joblib")

features_array = np.array(features)
label_array = np.array(label)

/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline


In [8]:
# me situo en el directorio
%cd "/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline"

# Levanto particiones
folds_ind = joblib.load('10foldsXRepeticion_W200_I50.pkl')


/content/drive/My Drive/repo_tesis/src/baseline/datos_baseline


In [25]:
for j in range(0, 10):

  # Creo una copia para no modificar el original
  folds_ind_copy = folds_ind.copy()
  print(len(folds_ind_copy))

  # Tengo que ir cambiando qué sujeto uso para test. El resto va todo para train
  folds_ind_test = folds_ind_copy[j]

  # Uno los datos de los folds de entrenamiento
  # me quedo con los folds de train quitando el de la posicion j
  folds_ind_copy.pop(j)
  folds_ind_train = folds_ind_copy.copy()

  # concateno los arrays (folds) en uno solo
  folds_ind_train_array = np.concatenate(folds_ind_train)
  # aplano el array resultante
  folds_ind_train_array = folds_ind_train_array.ravel()

  # Defino X_train, y_train, X_test, y_test
  X_train = features_array[folds_ind_train_array, :]
  y_train = label_array[folds_ind_train_array, 1]
  X_test = features_array[folds_ind_test, :]
  y_test = label_array[folds_ind_test, 1]

  start_time = time.time()    # comienzo a medir el tiempo
  # implemento gbm
  clf = LGBMClassifier()
  clf.fit(X_train, y_train, eval_metric='logloss')

  end_time = time.time()
  training_time = end_time - start_time
  # Obtener las iteraciones del mejor modelo
  bst = clf.best_iteration_

  nombre = 'baseline_gbm_sep_rep' + str(j+1) + '_test'

  joblib.dump(clf, nombre + '.pkl')

  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')
  conf_matrix = confusion_matrix(y_test, y_pred)
  plt.savefig("confusion_matrix.png")

  API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO'

  exp = Experiment(api_key=API_KEY,
                  project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                  auto_param_logging=False)
  exp.set_name(nombre) # Nombre de este experimento
  exp.add_tags(['baseline', 'gbm', 'sep_rep', 'choose_test']) # Tags

  exp.log_metric("accuracy", accuracy)
  exp.log_metric("precision", precision)
  exp.log_metric("recall", recall)
  exp.log_metric("training_time", training_time)
  exp.log_confusion_matrix(y_test, y_pred)
  exp.log_parameter("sujeto_test", j)   # Guarda el arreglo en el experimento
  exp.log_text("Se usa el archivo de particion de datos 10foldsXRepeticion_W200_I50.pkl. Se guarda como parametro el numero de fold que se usa para test (se usa solo 1)")   # Comentario del experimento
  exp.log_model(name=nombre, file_or_folder=nombre + '.pkl')
  exp.end()

10
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.383578 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50352
[LightGBM] [Info] Number of data points in the train set: 58175, number of used features: 240
[LightGBM] [Info] Start training from score -2.564949
[LightGBM] [Info] Start training from score -2.606237
[LightGBM] [Info] Start training from score -2.614891
[LightGBM] [Info] Start training from score -2.594430
[LightGBM] [Info] Start training from score -2.584810
[LightGBM] [Info] Start training from score -2.551851
[LightGBM] [Info] Start training from score -2.554944
[LightGBM] [Info] Start training from score -2.558712
[LightGBM] [Info] Start training from score -2.558268
[LightGBM] [Info] Start training from score -2.610438
[LightGBM] [Info] Start training from score -2.295647
[LightGBM] [Info] Start training from score -2.562049
[LightGBM] [Info] Start training from score -2.746244


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: lightgbm, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/738473bd2d2448d19d97fddddab59b0c

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_gbm_sep_rep1_test
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/738473bd2d2448d19d97fddddab59b0c
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.5750275547157928

10
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.335152 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50349
[LightGBM] [Info] Number of data points in the train set: 58169, number of used features: 240
[LightGBM] [Info] Start training from score -2.563730
[LightGBM] [Info] Start training from score -2.606600
[LightGBM] [Info] Start training from score -2.618553
[LightGBM] [Info] Start training from score -2.594327
[LightGBM] [Info] Start training from score -2.585391
[LightGBM] [Info] Start training from score -2.552631
[LightGBM] [Info] Start training from score -2.554841
[LightGBM] [Info] Start training from score -2.556169
[LightGBM] [Info] Start training from score -2.558831
[LightGBM] [Info] Start training from score -2.609400
[LightGBM] [Info] Start training from score -2.295203
[LightGBM] [Info] Start training from score -2.561055
[LightGBM] [Info] Start training from score -2.747212


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: lightgbm, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/82b07033824a4c4d851d4f7baddcbc12

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_gbm_sep_rep2_test
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/82b07033824a4c4d851d4f7baddcbc12
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.6059462010382256

10
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.198000 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50353
[LightGBM] [Info] Number of data points in the train set: 57950, number of used features: 240
[LightGBM] [Info] Start training from score -2.565105
[LightGBM] [Info] Start training from score -2.604927
[LightGBM] [Info] Start training from score -2.610781
[LightGBM] [Info] Start training from score -2.594013
[LightGBM] [Info] Start training from score -2.578658
[LightGBM] [Info] Start training from score -2.548417
[LightGBM] [Info] Start training from score -2.554837
[LightGBM] [Info] Start training from score -2.551290
[LightGBM] [Info] Start training from score -2.557728
[LightGBM] [Info] Start training from score -2.602828
[LightGBM] [Info] Start training from score -2.325450
[LightGBM] [Info] Start training from score -2.559065
[LightGBM] [Info] Start training from score -2.740228


...

10
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.202608 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50353
[LightGBM] [Info] Number of data points in the train set: 58185, number of used features: 240
[LightGBM] [Info] Start training from score -2.564898
[LightGBM] [Info] Start training from score -2.604548
[LightGBM] [Info] Start training from score -2.619064
[LightGBM] [Info] Start training from score -2.595753
[LightGBM] [Info] Start training from score -2.582250
[LightGBM] [Info] Start training from score -2.554673
[LightGBM] [Info] Start training from score -2.551803
[LightGBM] [Info] Start training from score -2.560439
[LightGBM] [Info] Start training from score -2.558218
[LightGBM] [Info] Start training from score -2.606176
[LightGBM] [Info] Start training from score -2.298212
[LightGBM] [Info] Start training from score -2.558440
[LightGBM] [Info] Start training from score -2.748561


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: lightgbm, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/68af6f2370b2478494d2bd23f5028266

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_gbm_sep_rep9_test
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/68af6f2370b2478494d2bd23f5028266
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.5647374231193818

10
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.195015 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50349
[LightGBM] [Info] Number of data points in the train set: 57979, number of used features: 240
[LightGBM] [Info] Start training from score -2.566503
[LightGBM] [Info] Start training from score -2.600768
[LightGBM] [Info] Start training from score -2.613868
[LightGBM] [Info] Start training from score -2.591516
[LightGBM] [Info] Start training from score -2.578022
[LightGBM] [Info] Start training from score -2.552676
[LightGBM] [Info] Start training from score -2.547815
[LightGBM] [Info] Start training from score -2.557560
[LightGBM] [Info] Start training from score -2.554893
[LightGBM] [Info] Start training from score -2.601466
[LightGBM] [Info] Start training from score -2.326834
[LightGBM] [Info] Start training from score -2.554449
[LightGBM] [Info] Start training from score -2.747702


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: lightgbm, sklearn.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/d0558e8e2eac407fb95a6c324ad768c3

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : baseline_gbm_sep_rep10_test
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/d0558e8e2eac407fb95a6c324ad768c3
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.572628684893844

<Figure size 640x480 with 0 Axes>

In [ ]:
start_time = time.time()    # comienzo a medir el tiempo

# Crear una instancia de LGBMClassifier
clf = LGBMClassifier()

# Entrenar el clasificador
clf.fit(X_train, y_train, eval_metric='logloss')

# Finalizo la medida del tiempo y calculo el tiempo de entrenamiento
end_time = time.time()
training_time = end_time - start_time

# # Realizar predicciones en el conjunto de validación
# y_pred = clf.predict(X_val)

# # Calcular la precisión
# accuracy = accuracy_score(y_val, y_pred)
# print(f'Accuracy: {accuracy}')

# Obtener las iteraciones del mejor modelo
best_iteration = clf.best_iteration_

# Imprimir la advertencia si existe
if hasattr(clf, 'best_iteration_') and clf.best_iteration_ is None:
    print("[LightGBM] [Warning] No further splits with positive gain, best gain: -inf")



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.216863 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50308
[LightGBM] [Info] Number of data points in the train set: 37655, number of used features: 240
[LightGBM] [Info] Start training from score -2.565481
[LightGBM] [Info] Start training from score -2.592793
[LightGBM] [Info] Start training from score -2.601708
[LightGBM] [Info] Start training from score -2.591020
[LightGBM] [Info] Start training from score -2.573805
[LightGBM] [Info] Start training from score -2.543614
[LightGBM] [Info] Start training from score -2.547339
[LightGBM] [Info] Start training from score -2.549036
[LightGBM] [Info] Start training from score -2.549036
[LightGBM] [Info] Start training from score -2.601708
[LightGBM] [Info] Start training from score -2.376846
[LightGBM] [Info] Start training from score -2.547339
[LightGBM] [Info] Start training from score -2.740986
Accurac

Voy a guardar el clasificador, y evaluar distintas métricas: accuracy, precision y recall. Voy a hacer una matriz de confusión.
Por otra parte, voy a guardar la partición de los datos para hacer reproducible el experimento.

In [ ]:
# Guardar el modelo entrenado en un archivo
joblib.dump(bst, 'baseline_gbm_sep_sub_r1.pkl')

# Predecir en el conjunto de test
y_pred = bst.predict(X_test)

# Calcular métricas de desempeño
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

# Mostrar las métricas
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

# Visualizar la matriz de confusión
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, cmap="Blues", fmt="d", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

# Guardar la imagen de la matriz de confusión
plt.savefig("confusion_matrix.png")

Voy a guardar las métricas calculadas en un experimento en Comet

In [ ]:
# Conectar con Comet
API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO' # Se puede encontrar en Settings(Arriba a la derecha en Comet)

# Crear un experimento con mi API KEY
exp = Experiment(api_key=API_KEY,
                 project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                 auto_param_logging=False)
exp.set_name('baseline_gbm_sep_sub_r1') # Nombre de este experimento
exp.add_tags(['baseline', 'gbm', 'sep_sub']) # Tags

exp.log_metric("accuracy", accuracy)
exp.log_metric("precision", precision)
exp.log_metric("recall", recall)
exp.log_metric("training_time", training_time)
exp.log_confusion_matrix(y_test, y_pred)
exp.log_parameter("partition_array", sorteo)   # Guarda el arreglo en el experimento
exp.log_text("Primeros dos sujetos --> test, tercero y cuarto --> validación, resto --> train. \n Corresponde a la primera ronda que entreno con estos parámetros. ")   # Comentario del experimento

In [ ]:
# Subir el modelo
exp.log_model(name="baseline_gbm_sep_sub_r1", file_or_folder="baseline_gbm_sep_sub_r1.pkl")
exp.end()

Junto todo en un bloque de código

In [ ]:
%cd "/content/drive/My Drive/repo_tesis/archivos_generados_codigos"

for k in range(1,2):
  label = np.array(label)
  cantSujetos = np.max(label[:, 2])
  sujeto_test = k
  sorteo = np.random.permutation(cantSujetos) + 1
  sorteo_sin_test = np.delete(sorteo, np.where(sorteo == sujeto_test))
  indices_test = list(np.where(label[:, 2]==sujeto_test)[0])
  indices_val = list(np.where(label[:, 2]==sorteo_sin_test[0])[0])
  indices_val.extend(list(np.where(label[:, 2]==sorteo_sin_test[1])[0]))

  indices_train = []
  for j in sorteo_sin_test[2:]:
      indices_train.extend(np.where(label[:, 2]==j)[0])
  features = np.array(features)

  X_train = features[indices_train, :]
  y_train = label[indices_train, 1]
  X_val = features[indices_val, :]
  y_val = label[indices_val, 1]
  X_test =  features[indices_test, :]
  y_test = label[indices_test, 1]

  start_time = time.time()

  # implemento gbm
  clf = LGBMClassifier()
  clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='logloss')

  end_time = time.time()
  training_time = end_time - start_time
  # Obtener las iteraciones del mejor modelo
  bst = clf.best_iteration_

  nombre = 'baseline_gbm_sep_sub' + str(k) + '_testing'

  joblib.dump(clf, nombre + '.pkl')

  y_pred = clf.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')
  conf_matrix = confusion_matrix(y_test, y_pred)
  plt.savefig("confusion_matrix.png")

  API_KEY = 'ehXeElNypcj7Knar5zTmyjwSO'

  exp = Experiment(api_key=API_KEY,
                  project_name='tesis-experimentos', # Nombre del proyecto donde se registran los experimentos
                  auto_param_logging=False)
  exp.set_name(nombre) # Nombre de este experimento
  exp.add_tags(['baseline', 'gbm', 'sep_sub', 'choose_test']) # Tags

  exp.log_metric("accuracy", accuracy)
  exp.log_metric("precision", precision)
  exp.log_metric("recall", recall)
  exp.log_metric("training_time", training_time)
  exp.log_confusion_matrix(y_test, y_pred)
  exp.log_parameter("partition_array", sorteo)   # Guarda el arreglo en el experimento
  exp.log_text("Se fija el sujeto de test y se sortea el resto. Los primeros dos sujetos del sorteo son los de validación.")   # Comentario del experimento
  exp.log_model(name=nombre, file_or_folder=nombre + '.pkl')
  exp.end()

/content/drive/My Drive/repo_tesis/archivos_generados_codigos
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.541994 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 50345
[LightGBM] [Info] Number of data points in the train set: 45226, number of used features: 240
[LightGBM] [Info] Start training from score -2.565215
[LightGBM] [Info] Start training from score -2.600029
[LightGBM] [Info] Start training from score -2.605103
[LightGBM] [Info] Start training from score -2.578238
[LightGBM] [Info] Start training from score -2.572718
[LightGBM] [Info] Start training from score -2.544725
[LightGBM] [Info] Start training from score -2.544161
[LightGBM] [Info] Start training from score -2.547263
[LightGBM] [Info] Start training from score -2.552927
[LightGBM] [Info] Start training from score -2.595573
[LightGBM] [Info] Start training from score -2.382079
[LightGBM] [Info] Start training from score -2.553779

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: sklearn, lightgbm.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/manuelmolinach99/tesis-experimentos/bd0cddec9e424fa4bd23561cf03d600f

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/manuelmolinach99/tesis-experimentos/bd0cddec9e424fa4bd23561cf03d600f
COMET INFO:   Metrics:
COMET INFO:     accuracy      : 0.25368837711406983
COMET INFO:     precision     : 0.26482242684700047
COMET INFO:  

<Figure size 640x480 with 0 Axes>